<a href="https://colab.research.google.com/github/JasonKilembe/Certificats/blob/master/BioActVec_Download_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chembl_webresource_client pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.0 MB/s eta 0:00:00


In [ ]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
import time

# Définir la liste des cibles ChEMBL (ex: HIF1A, AR, EGFR, mTOR, VEGFR2, TOP2A)
target_ids = {
    "HIF1A": "CHEMBL2221345",
    "AR": "CHEMBL1871",
    "EGFR": "CHEMBL203",
    "mTOR": "CHEMBL2842",
    "VEGFR2": "CHEMBL279",
    "TOP2A": "CHEMBL1824"
}

In [ ]:
# Fonction pour récupérer les bioactivités pour une cible
def get_bioactivities(target_id, assay_types=["IC50", "Ki", "EC50"], confidence_score=7):
    activities = new_client.activity.filter(
        target_chembl_id=target_id,
        standard_type__in=assay_types,
        standard_units="nM",
        standard_value__isnull=False,
        target_confidence_score__gte=confidence_score
    )
    return pd.DataFrame(activities)

In [ ]:

# Récupération des données pour chaque cible
all_data = []
for name, chembl_id in target_ids.items():
    print(f"Fetching data for {name} ({chembl_id})...")
    df = get_bioactivities(chembl_id)
    df["target_name"] = name
    all_data.append(df)
    time.sleep(1)  # pour éviter le blocage API

Fetching data for HIF1A (CHEMBL2221345)...
Fetching data for AR (CHEMBL1871)...
Fetching data for EGFR (CHEMBL203)...
Fetching data for mTOR (CHEMBL2842)...
Fetching data for VEGFR2 (CHEMBL279)...
Fetching data for TOP2A (CHEMBL1824)...


In [ ]:
# Concaténation des résultats
bioactivity_df = pd.concat(all_data, ignore_index=True)

# Garde les colonnes utiles
columns_to_keep = [
    'molecule_chembl_id', 'canonical_smiles', 'standard_type',
    'standard_value', 'standard_units', 'target_name'
]
bioactivity_df = bioactivity_df[columns_to_keep].dropna()

# Supprime les doublons
bioactivity_df.drop_duplicates(inplace=True)

# Enregistre les résultats en CSV
bioactivity_df.to_csv("bioactivity_data.csv", index=False)
print("Bioactivity data saved as 'bioactivity_data.csv'")

Bioactivity data saved as 'bioactivity_data.csv'
